In [3]:
import soundfile as sf
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor
from PIL import Image
import requests
from io import BytesIO

#음성인식 모델 및 프로세서 초기화
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h")

#음성파일을 불러오는 함수
def transcribe_audio(file_path):
    audio_input, _=sf.read(file_path)
    input_values = processor(audio_input, return_tensors="pt", padding="longest").input_values
    
    #음성인식
    with torch.no_grad():
        logits = model(input_values).logits
        
    #예측결과로부터 텍스트변환
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])
    return transcription

# 텍스트를 기반으로 이미지를 생성하는 함수
def generate_image_from_text(text):
    # DALL-E 또는 다른 이미지 생성 API를 호출
    url = f"https://api.huggingface.co/generation/dalle2/generate"
    headers = {"Authorization": f"HUGGINGFACE_API_KEY"}  #hugingkey
    payload = {"text": text}
    
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        image_url = response.json()["image"]
        return image_url
    else:
        return None

# 메인 실행 흐름
audio_file_path = "path/to/your/audio/file.wav"  # 음성 파일 경로
transcription = transcribe_audio(audio_file_path)
print("인식된 텍스트:", transcription)

image_url = generate_image_from_text(transcription)
if image_url:
    image_response = requests.get(image_url)
    img = Image.open(BytesIO(image_response.content))
    img.show()
else:
    print("이미지를 생성하는 데 실패했습니다.")
    
    
    

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LibsndfileError: Error opening 'path/to/your/audio/file.wav': System error.